# Lecture 9: Numpy Basics

NumPy（Numerical Python的简称）是Python数值计算最重要的基础包。大多数提供科学计算的包都是用NumPy的数组作为构建基础。

NumPy的部分功能如下：

- ndarray，一个具有矢量算术运算和复杂广播能力的快速且节省空间的多维数组。
- 用于对整组数据进行快速运算的标准数学函数（无需编写循环）。
- 用于读写磁盘数据的工具以及用于操作内存映射文件的工具。
- 线性代数、随机数生成以及傅里叶变换功能。
- 用于集成由C、C++、Fortran等语言编写的代码的A C API。

由于NumPy提供了一个简单易用的C API，因此很容易将数据传递给由低级语言编写的外部库，外部库也能以NumPy数组的形式将数据返回给Python。这个功能使Python成为一种包装C/C++/Fortran历史代码库的选择，并使被包装库拥有一个动态的、易用的接口。

NumPy本身并没有提供多么高级的数据分析功能，理解NumPy数组以及面向数组的计算将有助于你更加高效地使用诸如pandas之类的工具。因为NumPy是一个很大的题目，我会在下一节介绍更多NumPy高级功能，比如广播。

对于大部分数据分析应用而言，我们最关注的功能主要集中在：

- 用于数据整理和清理、子集构造和过滤、转换等快速的矢量化数组运算。
- 常用的数组算法，如排序、唯一化、集合运算等。
- 高效的描述统计和数据聚合/摘要运算。
- 用于异构数据集的合并/连接运算的数据对齐和关系型数据运算。
- 将条件逻辑表述为数组表达式（而不是带有if-elif-else分支的循环）。
- 数据的分组运算（聚合、转换、函数应用等）。

虽然NumPy提供了通用的数值数据处理的计算基础，但大多数读者可能还是想将pandas作为统计和分析工作的基础，尤其是处理表格数据时。pandas还提供了一些NumPy所没有的领域特定的功能，如时间序列处理等。

>笔记：Python的面向数组计算可以追溯到1995年，Jim Hugunin创建了Numeric库。接下来的10年，许多科学编程社区纷纷开始使用Python的数组编程，但是进入21世纪，库的生态系统变得碎片化了。2005年，Travis Oliphant从Numeric和Numarray项目整合出了NumPy项目，进而所有社区都集合到了这个框架下。

NumPy之于数值计算特别重要的原因之一，是因为它可以高效处理大数组的数据。这是因为：

- NumPy是在一个连续的内存块中存储数据，独立于其他Python内置对象。NumPy的C语言编写的算法库可以操作内存，而不必进行类型检查或其它前期工作。比起Python的内置序列，NumPy数组使用的内存更少。
- NumPy可以在整个数组上执行复杂的计算，而不需要Python的for循环。

要搞明白具体的性能差距，让我们来对比一个包含一百万整数的数组和一个等价的Python列表：

In [ ]:
import numpy as np

my_arr = np.arange(1000000)
my_list = list(range(1000000))

让它们分别乘以2：

In [ ]:
%timeit my_arr2 = my_arr * 2
%timeit my_list2 = [x * 2 for x in my_list]

基于NumPy的算法要比纯Python快10到100倍（甚至更快），并且使用的内存更少。

# NumPy的ndarray：一种多维数组对象
NumPy最重要的一个特点就是其N维数组对象（即ndarray），该对象是一个快速而灵活的大数据集容器。你可以利用这种数组对整块数据执行一些数学运算，其语法跟标量元素之间的运算一样。

我们来看一下Python是如何利用与标量值类似的语法进行批次计算的：

In [ ]:
data = np.array([[1.5, -0.1, 3], [0, -3, 6.5]])
data

In [ ]:
data * 10

In [ ]:
data + data

ndarray是一个通用的同构数据多维容器，也就是说，其中的所有元素必须是相同类型的。每个数组都有一个shape（一个表示各维度大小的元组）和一个dtype（一个用于说明数组数据类型的对象）：

In [ ]:
print(data.shape)
print(data.dtype)

## 创建ndarray数组
创建数组最简单的办法就是使用array函数。它接受一切序列型的对象（包括其他数组），然后产生一个新的含有传入数据的NumPy数组。以一个列表的转换为例：

In [ ]:
data1 = [6, 7.5, 8, 0, 1]
arr1 = np.array(data1)
arr1

In [ ]:
arr1.shape

嵌套序列（比如由一组等长列表组成的列表）将会被转换为一个多维数组：

In [ ]:
data2 = [[1, 2, 3, 4], [5, 6, 7, 8]]
arr2 = np.array(data2)
arr2

In [ ]:
data3 = [[[1, 2, 3, 4], [5, 6, 7, 8]], [[1, 2, 3, 4], [5, 6, 7, 8]]]
arr3 = np.array(data3)
arr3

In [ ]:
arr3.shape

In [ ]:
print(arr2.ndim)
print(arr2.shape)

除非特别说明（稍后将会详细介绍），np.array会尝试为新建的这个数组推断出一个较为合适的数据类型。数据类型保存在一个特殊的dtype对象中。比如说，在上面的两个例子中，我们有：

In [ ]:
print(arr1.dtype)
print(arr2.dtype)

除np.array之外，还有一些函数也可以新建数组。比如，zeros和ones分别可以创建指定长度或形状的全0或全1数组。empty可以创建一个没有任何具体值的数组。要用这些方法创建多维数组，只需传入一个表示形状的元组即可：

In [ ]:
np.zeros(10)

In [ ]:
np.zeros((3, 6))

In [ ]:
np.empty((2, 3, 2))

In [ ]:
np.ones(10)

arange是Python内置函数range的数组版：

In [ ]:
list(range(15))

In [ ]:
np.arange(15)

In [ ]:
np.ones_like(list(range(15)))

表4-1列出了一些数组创建函数。由于NumPy关注的是数值计算，因此，如果没有特别指定，数据类型基本都是float64（浮点数）。

<img src="figures/creating_array.png" alt="creating_array" width="800"/>

## ndarray的数据类型
数值型dtype的命名方式相同：一个类型名（如float或int），后面跟一个用于表示各元素位长的数字。标准的双精度浮点值（即Python中的float对象）需要占用8字节（即64位）。

In [ ]:
arr1 = np.array([1, 2, 3], dtype=np.float64)
arr2 = np.array([1, 2, 3], dtype=np.int32)
print(arr1.dtype)
print(arr2.dtype)

In [ ]:
arr1

In [ ]:
arr2

<font color='limegreen'><b>Tips and Tricks</b></font>\
记不住这些NumPy的dtype也没关系。通常只需要知道你所处理的数据的大致类型是浮点数、复数、整数、布尔值、字符串，还是普通的Python对象即可。当你需要控制数据在内存和磁盘中的存储方式时（尤其是对大数据集），那就得了解如何控制存储类型。

你可以通过**astype**方法将数组从一个dtype转换成另一个dtype：

In [ ]:
arr = np.array([1, 2, 3, 4, 5])
print(arr)
print(arr.dtype)

In [ ]:
float_arr = arr.astype(np.float64)

print(float_arr)
print(float_arr.dtype)

如果将浮点数转换成整数，则小数部分将会被截取删除：

In [ ]:
arr = np.array([3.7, -1.2, -2.6, 0.5, 12.9, 10.1])
arr

In [ ]:
arr.astype(np.int32)

## NumPy数组的运算
数组很重要，因为它使你不用编写循环即可对数据执行批量运算。NumPy用户称其为矢量化（vectorization）。大小相等的数组之间的任何算术运算都会将运算应用到元素级：

In [ ]:
arr = np.array([[1., 2., 3.], [4., 5., 6.]])
print(arr)

In [ ]:
print(arr * arr)
print(arr - arr)

数组与标量的算术运算会将标量值传播(broadcast)到各个元素：

In [ ]:
print(1 / arr)
print(arr ** 3)

大小相同的数组之间的比较会生成布尔值数组：

In [ ]:
arr2 = np.array([[0., 4., 1.], [7., 2., 12.]])
arr2

In [ ]:
arr2 > arr

## 基本的索引和切片
NumPy数组的索引是一个内容丰富的主题，因为选取数据子集或单个元素的方式有很多。\
一维数组很简单。从表面上看，它们跟Python列表的功能差不多：

In [ ]:
arr = np.arange(10)
arr

In [ ]:
arr[5:8] = 12

In [ ]:
arr

In [ ]:
arr[5]
arr[5:8]

arr[5:8] = 12
arr

跟列表最重要的区别在于，数组切片是原始数组的视图。这意味着数据不会被复制，视图上的任何修改都会直接反映到源数组上。\
作为例子，先创建一个arr的切片：

In [ ]:
arr_slice = arr[5:8]
arr_slice

现在，当我修改arr_slice中的值，变动也会体现在原始数组arr中：

In [ ]:
arr_slice[1] = 12345
arr

<font color='limegreen'><b>Tips and Tricks</b></font>\
为什么呢？\
由于NumPy的设计目的是处理大数据，所以你可以想象一下，假如NumPy坚持要将数据复制来复制去的话会产生何等的性能和内存问题。

如果你想要得到的是ndarray切片的一份副本而非视图，就需要明确地进行复制操作，例如``arr[5:8].copy()``。

In [ ]:
arr_copy = arr[5:8].copy()
arr_copy[1] = 54321
arr

In [ ]:
arr_copy

对于高维度数组，能做的事情更多。在一个二维数组中，各索引位置上的元素不再是标量而是一维数组：

In [ ]:
arr2d = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(arr2d)
print(arr2d.shape)
print(arr2d[2])

如果索引第2个维度，用“，”隔开：

In [ ]:
arr2d[0, 2]

In [ ]:
arr2d[1,2]

下图说明了二维数组的索引方式。轴0作为行，轴1作为列。
![numpy_axis](figures/numpy_axis.jpg)

在多维数组中，如果省略了后面的索引，则返回对象会是一个维度低一点的ndarray（它含有高一级维度上的所有数据）。因此，在2×2×3数组arr3d中：

In [ ]:
arr3d = np.array([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]])
arr3d

In [ ]:
arr3d.shape

In [ ]:
arr3d[0].shape

标量值和数组都可以被赋值给arr3d[0]：

In [ ]:
old_values = arr3d[0].copy()
arr3d[0] = 42
print(arr3d)
arr3d[0] = old_values
print(arr3d)

相似的，arr3d[1,0]可以访问索引以(1,0)开头的那些值（以一维数组的形式返回）：

In [ ]:
arr3d[1, 0, 2]

### 切片索引
ndarray的切片语法跟Python列表的切片类似：

In [ ]:
arr2d

In [ ]:
arr2d[:2]

In [ ]:
arr2d[:2, 1:]

In [ ]:
arr2d[:2, 2]

In [ ]:
arr2d[:, :1]

In [ ]:
arr2d[:2, 1:] = 0
arr2d

## 布尔型索引
来看这样一个例子，假设我们有一个用于存储数据的数组以及一个存储姓名的数组（含有重复项）。在这里，我将使用numpy.random中的randn函数生成一些正态分布的随机数据：

In [ ]:
data = np.random.randn(7, 4)
data

In [ ]:
names = np.array(["Bob", "Joe", "Will", "Bob", "Will", "Joe", "Joe"])
names

如果我们想要选出对应于名字"Bob"的所有行：

In [ ]:
names == "Bob"

In [ ]:
data[names == "Bob", :]

布尔型数组的长度必须跟被索引的轴长度一致。此外，还可以将布尔型数组跟切片、整数（或整数序列，稍后将对此进行详细讲解）混合使用：

In [ ]:
data[names == "Bob", 1:]

In [ ]:
data[names == "Bob", 1]

要选择除"Bob"以外的其他值，既可以使用不等于符号（!=），也可以通过~对条件进行否定：

In [ ]:
names != "Bob"

In [ ]:
~(names == "Bob")

In [ ]:
data[~(names == "Bob")]

选取这三个名字中的两个需要组合应用多个布尔条件，使用&（和）、|（或）之类的布尔算术运算符即可：

In [ ]:
mask = (names == "Bob") | (names == "Will")
data[mask]

通过布尔型数组设置值是一种经常用到的手段。为了将data中的所有负值都设置为0，我们只需：

In [ ]:
data < 0

In [ ]:
data[data < 0] = 0
data

通过一维布尔数组设置整行或列的值也很简单：

In [ ]:
data[names != "Joe"] = 7
data

后面会看到，这类二维数据的操作也可以用pandas方便的来做。

## 花式索引
花式索引（Fancy indexing）是一个NumPy术语，它指的是利用整数数组进行索引。假设我们有一个8×4数组：

In [ ]:
arr = np.zeros((8, 4))
for i in range(8):
    arr[i] = i
arr

为了以特定顺序选取行子集，只需传入一个用于指定顺序的整数列表或ndarray即可：

In [ ]:
arr[[4, 3, 0, 6]]

使用负数索引将会从末尾开始选取行：

In [ ]:
arr[[-3, -5, -7]]

一次传入多个索引数组会有一点特别。它返回的是一个一维数组，其中的元素对应各个索引元组：

In [ ]:
arr = np.arange(32).reshape((8, 4))
arr

In [ ]:
arr[[1, 5, 7, 2], [0, 3, 1, 2]]

最终选出的是元素(1,0)、(5,3)、(7,1)和(2,2)。无论数组是多少维的，花式索引总是一维的。

这跟我们的预期不一样，通常我们认为选取矩阵的行列子集应该是矩形区域的形式才对。下面是得到该结果的一个办法：

In [ ]:
arr[[1, 5, 7, 2]][:, [0, 3, 1, 2]]

## 数组转置和轴对换
转置是重塑的一种特殊形式，它返回的是源数据的视图（不会进行任何复制操作）。数组不仅有transpose方法，还有一个特殊的T属性：

In [ ]:
arr = np.arange(15).reshape((3, 5))
arr

In [ ]:
arr.shape

In [ ]:
arr.T

In [ ]:
arr.T.shape

例如，在进行矩阵计算时，经常需要用到该操作，比如利用np.dot计算矩阵内积：

In [ ]:
arr = np.random.randn(6, 3)
arr

In [ ]:
arr.T.shape

In [ ]:
arr.shape

In [ ]:
np.dot(arr.T, arr)

对于高维数组，transpose需要得到一个由轴编号组成的元组才能对这些轴进行转置（比较费脑子）：

In [ ]:
arr = np.arange(24).reshape((2, 3, 4))
arr

In [ ]:
arr.shape

In [ ]:
arr_t = arr.transpose((2, 0, 1))
arr_t

In [ ]:
# 这里，第一个轴被换成了第二个，第二个轴被换成了第一个，最后一个轴不变。
arr_t.shape

简单的转置可以使用.T，它其实就是进行轴对换而已。ndarray还有一个swapaxes方法，它需要接受一对轴编号：

In [ ]:
arr.swapaxes(1, 2).shape

## 通用函数：快速的元素级数组函数
通用函数（即ufunc）是一种对ndarray中的数据执行元素级运算的函数。你可以将其看做简单函数（接受一个或多个标量值，并产生一个或多个标量值）的矢量化包装器。

许多ufunc都是简单的元素级变体，如sqrt和exp：

In [ ]:
arr = np.arange(10)
arr

In [ ]:
np.sqrt(arr)

In [ ]:
np.exp(arr) # e^x

numpy.maximum计算了x和y中元素级别最大的元素。

In [ ]:
x = np.random.randn(8)
y = np.random.randn(8)
print(x)
print(y)

In [ ]:
print(np.maximum(x, y))

![numpy_ufunc1](figures/numpy_ufunc1.jpg)
![numpy_ufunc2](figures/numpy_ufunc2.jpg)
![numpy_ufunc3](figures/numpy_ufunc3.jpg)

# 利用数组进行数据处理
NumPy数组使你可以将许多种数据处理任务表述为简洁的数组表达式（否则需要编写循环）。用数组表达式代替循环的做法，通常被称为矢量化。一般来说，矢量化数组运算要比等价的纯Python方式快上一两个数量级（甚至更多），尤其是各种数值计算。

举个简单的例子，假设我们想要在一组值（网格型）上计算函数sqrt(x^2+y^2)。np.meshgrid函数接受两个一维数组，并产生两个二维矩阵（对应于两个数组中所有的(x,y)对）：

In [ ]:
points = np.arange(-5, 5, 0.01) # 100 equally spaced points
xs, ys = np.meshgrid(points, points)

In [ ]:
xs

In [ ]:
ys

In [ ]:
z = np.sqrt(xs ** 2 + ys ** 2)
z

我用matplotlib创建了这个二维数组的可视化：

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(z, cmap=plt.cm.gray, extent=[-5, 5, -5, 5])
plt.colorbar()
plt.title("Image plot of $\sqrt{x^2 + y^2}$ for a grid of values")

## 将条件逻辑表述为数组运算
numpy.where函数是三元表达式x if condition else y的矢量化版本。假设我们有一个布尔数组和两个值数组：

In [ ]:
xarr = np.array([1.1, 1.2, 1.3, 1.4, 1.5])
yarr = np.array([2.1, 2.2, 2.3, 2.4, 2.5])
cond = np.array([True, False, True, True, False])

假设我们想要根据cond中的值选取xarr和yarr的值：当cond中的值为True时，选取xarr的值，否则从yarr中选取。列表推导式的写法应该如下所示：

In [ ]:
result = np.zeros(len(cond))

for i in range(len(cond)):
    if cond[i] == True:
        result[i] = xarr[i]
    else:
        result[i] = yarr[i]

result

In [ ]:
result = [(x if c else y)
          for x, y, c in zip(xarr, yarr, cond)]
result

这有几个问题。第一，它对大数组的处理速度不是很快（因为所有工作都是由纯Python完成的）。第二，无法用于多维数组。若使用np.where，则可以将该功能写得非常简洁：

In [ ]:
result = np.where(cond, xarr, yarr)
result

np.where的第二个和第三个参数不必是数组，它们都可以是标量值。在数据分析工作中，where通常用于根据另一个数组而产生一个新的数组。假设有一个由随机数据组成的矩阵，你希望将所有正值替换为2，将所有负值替换为－2。若利用np.where，则会非常简单：

In [ ]:
arr = np.random.randn(4, 4)
arr

In [ ]:
np.where(arr > 0, 2, -2)

使用np.where，可以将标量和数组结合起来。例如，我可用常数2替换arr中所有正的值：

In [ ]:
np.where(arr > 0, 2, arr) # set only positive values to 2

## 数学和统计方法
可以通过数组上的一组数学函数对整个数组或某个轴向的数据进行统计计算。sum、mean以及标准差std等聚合计算（aggregation，通常叫做约简（reduction））既可以当做数组的实例方法调用，也可以当做顶级NumPy函数使用。

首先，我们生成了一些正态分布随机数据，然后做了聚类统计：

In [ ]:
arr = np.random.rand(5,4)
arr

In [ ]:
arr.mean()

In [ ]:
np.mean(arr)

In [ ]:
arr.sum()

mean和sum这类的函数可以接受一个axis选项参数，用于计算该轴向上的统计值，最终结果是一个少一维的数组：

In [ ]:
arr.mean(axis=1) # 计算行的平均值

In [ ]:
arr.sum(axis=0) # 计算列的和

其他如cumsum和cumprod之类的方法则不聚合，而是产生一个由中间结果组成的数组：

In [ ]:
arr = np.array([0, 1, 2, 3, 4, 5, 6, 7])
arr.cumsum()

在多维数组中，累加函数（如cumsum）返回的是同样大小的数组，但是会根据每个低维的切片沿着标记轴计算部分聚类：

In [ ]:
arr = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
arr

In [ ]:
arr.cumsum(axis=0)

In [ ]:
arr.cumprod(axis=1)

下表列出了全部的基本数组统计方法。
![numpy_methods](figures/numpy_methods.png)
![numpy_methods2](figures/numpy_methods2.png)

## 用于布尔型数组的方法
在上面这些方法中，布尔值会被强制转换为1（True）和0（False）。因此，sum经常被用来对布尔型数组中的True值计数：

In [ ]:
arr = np.random.randn(100)
arr

In [ ]:
(arr > 0).sum() # Number of positive values

In [ ]:
(arr <= 0).sum() # Number of non-positive values

另外还有两个方法any和all，它们对布尔型数组非常有用。any用于测试数组中是否存在一个或多个True，而all则检查数组中所有值是否都是True：

In [ ]:
bools = np.array([False, False, True, False])
bools.any()

In [ ]:
bools.all()

## 排序
跟Python内置的列表类型一样，NumPy数组也可以通过sort方法就地排序：

In [ ]:
arr = np.random.randn(6)
arr

In [ ]:
arr.sort()
arr

多维数组可以在任何一个轴向上进行排序，只需将轴编号传给sort即可：

In [ ]:
arr = np.random.randn(5, 3)
arr

In [ ]:
arr.sort(axis=0)
arr

In [ ]:
arr.sort(axis=1)
arr

注意：顶级函数np.sort返回的是数组的已排序副本，而就地排序则会修改数组本身。

In [ ]:
arr2 = np.array([5, -10, 7, 1, 0, -3])
sorted_arr2 = np.sort(arr2)
sorted_arr2

In [ ]:
arr2

计算数组分位数最简单的办法是对其进行排序，然后选取特定位置的值：

In [ ]:
large_arr = np.random.randn(1000)
large_arr.sort()
large_arr[int(0.05 * len(large_arr))] # 5% quantile

唯一化以及其它的集合逻辑
NumPy提供了一些针对一维ndarray的基本集合运算。最常用的可能要数np.unique了，它用于找出数组中的唯一值并返回已排序的结果：

In [ ]:
names = np.array(["Bob", "Will", "Joe", "Bob", "Will", "Joe", "Joe"])
np.unique(names)

In [ ]:
ints = np.array([3, 3, 3, 2, 2, 1, 1, 4, 4])
np.unique(ints)

拿跟np.unique等价的纯Python代码来对比一下：

In [ ]:
sorted(set(names))

另一个函数np.in1d用于测试一个数组中的值在另一个数组中的成员资格，返回一个布尔型数组：

In [ ]:
values = np.array([6, 0, 0, 3, 2, 5, 6])
np.in1d(values, [2, 3, 6])

## 用于数组的文件输入输出
NumPy能够读写磁盘上的文本数据或二进制数据。这一小节只讨论NumPy的内置二进制格式，因为更多的用户会使用pandas或其它工具加载文本或表格数据（见下一章）。

np.save和np.load是读写磁盘数组数据的两个主要函数。默认情况下，数组是以未压缩的原始二进制格式保存在扩展名为.npy的文件中的：

In [ ]:
arr = np.arange(10)
np.save("some_array", arr)

如果文件路径末尾没有扩展名.npy，则该扩展名会被自动加上。然后就可以通过np.load读取磁盘上的数组：

In [ ]:
np.load("some_array.npy")

## 线性代数
线性代数（如矩阵乘法、矩阵分解、行列式以及其他方阵数学等）是任何数组库的重要组成部分。不像某些语言（如MATLAB），通过*对两个二维数组相乘得到的是一个元素级的积，而不是一个矩阵点积。因此，NumPy提供了一个用于矩阵乘法的dot函数（既是一个数组方法也是numpy命名空间中的一个函数）：

In [ ]:
x = np.array([[1., 2., 3.], [4., 5., 6.]])
y = np.array([[6., 23.], [-1, 7], [8, 9]])
print(x)
print(y)

In [ ]:
x.dot(y)

In [ ]:
np.dot(x, y) # 等价于x.dot(y)

## 伪随机数生成
numpy.random模块对Python内置的random进行了补充，增加了一些用于高效生成多种概率分布的样本值的函数。例如，你可以用normal来得到一个标准正态分布的4×4样本数组：

In [ ]:
samples = np.random.normal(size=(4, 4))
samples

而Python内置的random模块则只能一次生成一个样本值。从下面的测试结果中可以看出，如果需要产生大量样本值，numpy.random快了不止一个数量级：

In [ ]:
from random import normalvariate
N = 1000000

%timeit samples = [normalvariate(0, 1) for _ in range(N)]

In [ ]:
%timeit np.random.normal(size=N)

我们说这些都是伪随机数，是因为它们都是通过算法基于随机数生成器种子，在确定性的条件下生成的。你可以用NumPy的np.random.seed更改随机数生成种子：

In [ ]:
np.random.seed(1234)

In [ ]:
np.random.randn(5)

下表列出了numpy.random中的部分函数。
![numpy_random](figures/numpy_random.png)
![numpy_random2](figures/numpy_random2.png)

## 示例：随机漫步
我们通过模拟随机漫步来说明如何运用数组运算。先来看一个简单的随机漫步的例子：从0开始，步长1和－1出现的概率相等。

下面是一个通过内置的random模块以纯Python的方式实现1000步的随机漫步：

In [ ]:
import random
position = 0
walk = [position]
nsteps = 1000
for i in range(nsteps):
    step = 1 if random.randint(0, 1) else -1
    position += step
    walk.append(position)


In [ ]:
plt.plot(walk[:100])

不难看出，这其实就是随机漫步中各步的累计和，可以用一个数组运算来实现。因此，我用np.random模块一次性随机产生1000个“掷硬币”结果（即两个数中任选一个），将其分别设置为1或－1，然后计算累计和：

In [ ]:
nsteps = 1000
draws = np.random.randint(0, 2, size=nsteps)
steps = np.where(draws > 0, 1, -1)
walk = steps.cumsum()

In [ ]:
plt.plot(walk)

有了这些数据之后，我们就可以沿着漫步路径做一些统计工作了，比如求取最大值和最小值：

In [ ]:
walk.min()

In [ ]:
walk.max()

现在来看一个复杂点的统计任务——首次穿越时间，即随机漫步过程中第一次到达某个特定值的时间。假设我们想要知道本次随机漫步需要多久才能距离初始0点至少10步远（任一方向均可）。np.abs(walk)>=10可以得到一个布尔型数组，它表示的是距离是否达到或超过10，而我们想要知道的是第一个10或－10的索引。可以用argmax来解决这个问题，它返回的是该布尔型数组第一个最大值的索引（True就是最大值）：

In [ ]:
(np.abs(walk) >= 10).argmax()

## 一次模拟多个随机漫步
如果你希望模拟多个随机漫步过程（比如5000个），只需对上面的代码做一点点修改即可生成所有的随机漫步过程。只要给numpy.random的函数传入一个二元元组就可以产生一个二维数组，然后我们就可以一次性计算5000个随机漫步过程（一行一个）的累计和了：

In [ ]:
nwalks = 5000
nsteps = 1000
draws = np.random.randint(0, 2, size=(nwalks, nsteps)) # 0 or 1
steps = np.where(draws > 0, 1, -1)
walks = steps.cumsum(axis=1)
walks

In [ ]:
walks.max()

In [ ]:
walks.min()

得到这些数据之后，我们来计算30或－30的最小穿越时间。这里稍微复杂些，因为不是5000个过程都到达了30。我们可以用any方法来对此进行检查：

In [ ]:
hits30 = (np.abs(walks) >= 30).any(axis=1)
hits30

In [ ]:
hits30.sum() # Number that hit 30 or -30, the total walks is 5000 times

In [ ]:
crossing_times = (np.abs(walks[hits30]) >= 30).argmax(axis=1)
crossing_times

In [ ]:
crossing_times.mean()

## The End! :)